In [3]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import simpledialog
from scipy import stats
import statsmodels.api as sm

In [26]:
# 분석할 이미지들이 있는 디렉토리 설정 (GUI에서 사용자로부터 입력받기)
root = tk.Tk()
root.withdraw()
directory_path = filedialog.askdirectory(title="분석할 이미지들이 있는 디렉토리를 선택하세요")

if not directory_path:
    messagebox.showerror("오류", "이미지 디렉토리를 선택하지 않았습니다.")
    exit()

# 결과 저장을 위한 리스트 초기화
ctrl_results = []
sample_results = []
other_results = []

# 결과 이미지 저장 디렉토리 설정 및 생성
output_directory = filedialog.askdirectory(title="결과 이미지를 저장할 디렉토리를 선택하세요")
if not output_directory:
    messagebox.showerror("오류", "결과 디렉토리를 선택하지 않았습니다.")
    exit()

segmentation_output_directory = os.path.join(output_directory, "segmentation_images")
os.makedirs(segmentation_output_directory, exist_ok=True)

# 디렉토리 내 모든 이미지 파일 반복 처리
for filename in os.listdir(directory_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.tif')):
        # 이미지 로드
        image_path = os.path.join(directory_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

        # 이미지 로드 확인
        if image is None:
            try:
                # Pillow를 사용하여 TIFF 이미지 로드 시도
                image_pil = Image.open(image_path)
                image = np.array(image_pil)

                # Grayscale 이미지를 RGB로 변환
                if len(image.shape) == 2:
                    image = cv2.merge([image, image, image])
            except Exception as e:
                print(f"이미지를 로드할 수 없습니다: {filename}, 오류: {e}")
                continue

        # BGR에서 RGB로 변환 (OpenCV는 기본적으로 BGR 형식으로 이미지를 읽음)
        if len(image.shape) == 3 and image.shape[2] == 3:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            image_rgb = cv2.merge([image, image, image])  # Grayscale 이미지를 RGB로 변환

        # 원본 이미지 저장
        original_image_path = os.path.join(segmentation_output_directory, f"{os.path.splitext(filename)[0]}_original.png")
        plt.imsave(original_image_path, image_rgb)

        # 이미지 전처리 - 대비 조정 (CLAHE 사용)
        lab = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        lab = cv2.merge((cl, a, b))
        image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

        # RGB를 HSV로 변환
        image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        # HSV 값으로 Oil Red O의 빨간색 범위 지정 (두 개의 빨간색 범위를 사용)
        lower_red1 = np.array([0, 100, 50])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([170, 100, 50])
        upper_red2 = np.array([180, 255, 255])

        # 빨간색 부분을 마스크로 추출
        mask1 = cv2.inRange(image_hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(image_hsv, lower_red2, upper_red2)
        mask = mask1 | mask2

        # Morphological 연산을 사용하여 마스크 개선
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.dilate(mask, kernel, iterations=2)

        # Segmentation된 마스크 이미지 저장
        mask_image_path = os.path.join(segmentation_output_directory, f"{os.path.splitext(filename)[0]}_mask.png")
        plt.imsave(mask_image_path, mask, cmap='gray')

        # 마스크 적용하여 염색된 부분만 남기기
        result = cv2.bitwise_and(image_rgb, image_rgb, mask=mask)

        # Masked 이미지 저장
        masked_image_path = os.path.join(segmentation_output_directory, f"{os.path.splitext(filename)[0]}_masked.png")
        plt.imsave(masked_image_path, result)

        # 염색된 부분의 픽셀 수 계산
        red_pixels = cv2.countNonZero(mask)

        # 전체 픽셀 수 계산
        total_pixels = image_rgb.shape[0] * image_rgb.shape[1]

        # 염색된 부분의 비율 계산
        red_ratio = (red_pixels / total_pixels) * 100 if total_pixels > 0 else 0

        # 그룹에 따라 결과 저장
        if 'ctrl' in filename.lower():
            ctrl_results.append({'Filename': filename, 'Stained Area Ratio (%)': red_ratio})
        elif 'sample' in filename.lower():
            sample_results.append({'Filename': filename, 'Stained Area Ratio (%)': red_ratio})
        else:
            other_results.append({'Filename': filename, 'Stained Area Ratio (%)': red_ratio})

        # 결과 출력
        print(f"{filename} - 염색된 지방의 비율: {red_ratio:.2f}%")

# 결과를 DataFrame으로 변환
ctrl_results_df = pd.DataFrame(ctrl_results)
sample_results_df = pd.DataFrame(sample_results)
other_results_df = pd.DataFrame(other_results)

# 엑셀 파일 저장 경로 설정
excel_path = os.path.join(output_directory, "stained_area_results_grouped.xlsx")

# 엑셀 파일 작성 엔진을 명시적으로 지정
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    if not ctrl_results_df.empty:
        ctrl_results_df.to_excel(writer, sheet_name='Control', index=False)
    if not sample_results_df.empty:
        sample_results_df.to_excel(writer, sheet_name='Sample', index=False)
    if not other_results_df.empty:
        other_results_df.to_excel(writer, sheet_name='Other', index=False)
    
    if ctrl_results_df.empty and sample_results_df.empty and other_results_df.empty:
        # 최소한 하나의 시트를 빈 시트로 작성
        pd.DataFrame().to_excel(writer, sheet_name='Empty')

print(f"모든 이미지의 분석 결과가 {excel_path}에 저장되었습니다.")

08.28_ITO1_20x-2.tif - 염색된 지방의 비율: 1.08%
08.28_oleic 0.05_20x-2.tif - 염색된 지방의 비율: 0.00%
08.28_oleic 0.1_20x-2.tif - 염색된 지방의 비율: 0.04%
08.28_oleic 0.2_20x-2.tif - 염색된 지방의 비율: 0.28%
모든 이미지의 분석 결과가 C:/Users/tissenbiofarm/Desktop/HN23SF01P/240807\stained_area_results_grouped.xlsx에 저장되었습니다.


In [13]:
ctrl_results_df.head()

,Filename,Stained Area Ratio (%)
0,Ctrl (1).tiff,30.900860
1,Ctrl (2).tiff,32.443810
2,Ctrl (3).tiff,46.705055
3,Ctrl (4).tiff,35.833836
4,Ctrl (5).tiff,31.632964


In [24]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np

# 결과 엑셀 파일 불러오기
excel_path = filedialog.askopenfilename(
    title = '분석할 엑셀 파일을 선택하세요',
    filetypes = [('Excel files', '*xlsx *xls')] # 파일 형식 필터링
)

# 엑셀 파일 경로가 선택되지 않은 경우 오류 메시지를 띄운다
if not excel_path:
    messagebox.showerror('오류', '엑셀 파일을 선택하지 않았습니다.')
    exit()

try:
    df = pd.read_excel(excel_path, sheet_name = None)
    # 필요한 데이터 분석 작업을 여기서 수행
    print('엑셀 파일이 성공적으로 로드하였습니다.')
except FileNotFoundError:
    messagebox.showerror('오류', '엑셀 파일을 찾을 수 없습니다.')
    exit()

# 각 그룹의 데이터 준비
ctrl_results_df = df.get('Control', pd.DataFrame())
sample_results_df = df.get('Sample', pd.DataFrame())
other_results = {key: value for key, value in df.items() if key not in ['Control', 'Sample']}

# 정규성 및 유의성 검증 결과 저장을 위한 데이터프레임 준비
normality_results = []
significance_results = []
summary_results = []

# 정규성 검증 (Shapiro-Wilk Test)
grouped_data = {'Control': ctrl_results_df, 'Sample': sample_results_df}
grouped_data.update(other_results)

all_data = []
group_labels = []

# 실제로 'Stained Area Ratio (%)' 컬럼을 가진 그룹들만 포함
valid_groups = {}

for group_name, group_df in grouped_data.items():
    if not group_df.empty and 'Stained Area Ratio (%)' in group_df.columns:
        data = group_df['Stained Area Ratio (%)']

        # n 수가 적을 때 데이터 증폭 (Bootstrap 샘플링)
        if len(data) < 30:
            data = np.random.choice(data, size=30, replace=True)

        # 데이터와 그룹명을 저장하여 ANOVA에 사용
        all_data.extend(data)
        group_labels.extend([group_name] * len(data))

        # 정규성 검증
        stat, p_value = stats.shapiro(data)
        normality_results.append({'Group': group_name, 'Shapiro-Wilk p-value': p_value})
        summary_results.append({'Group': group_name, 'Normality': 'Pass' if p_value > 0.05 else 'Fail'})
        valid_groups[group_name] = data
        print(f"{group_name} 그룹의 정규성 검증 (Shapiro-Wilk Test): p-value = {p_value:.4f}")
        if p_value > 0.05:
            print(f"{group_name} 그룹은 정규 분포를 따릅니다. (p > 0.05)")
        else:
            print(f"{group_name} 그룹은 정규 분포를 따르지 않습니다. (p <= 0.05)")

# 그룹이 2개인 경우와 3개 이상인 경우로 나누어 유의성 검증 수행
if len(valid_groups) == 2:
    # 그룹이 2개인 경우 t-test 또는 Mann-Whitney U Test 수행
    group_names = list(valid_groups.keys())
    group1_name = group_names[0]
    group2_name = group_names[1]
    group1_data = valid_groups[group1_name]
    group2_data = valid_groups[group2_name]

    # n 수가 적을 때 데이터 증폭 (Bootstrap 샘플링)
    if len(group1_data) < 30:
        group1_data = np.random.choice(group1_data, size=30, replace=True)
    if len(group2_data) < 30:
        group2_data = np.random.choice(group2_data, size=30, replace=True)

    # 정규성 여부에 따라 다른 검증 방법 사용
    group1_stat, group1_p = stats.shapiro(group1_data)
    group2_stat, group2_p = stats.shapiro(group2_data)

    if group1_p > 0.05 and group2_p > 0.05:
        # 두 그룹 모두 정규성을 따르는 경우 t-test 사용
        t_stat, t_p_value = stats.ttest_ind(group1_data, group2_data)
        significance_results.append({'Group 1': group1_name, 'Group 2': group2_name, 'Test': 't-test', 'p-value': t_p_value})
        summary_results.append({'Group 1': group1_name, 'Group 2': group2_name, 'Significance Test': 't-test', 'p-value': t_p_value, 'Significance': 'No' if t_p_value > 0.05 else 'Yes'})
        print(f"{group1_name}와 {group2_name} 간 t-test 결과: p-value = {t_p_value:.4f}")
    else:
        # 두 그룹 중 하나라도 정규성을 따르지 않는 경우 Mann-Whitney U Test 사용
        u_stat, u_p_value = stats.mannwhitneyu(group1_data, group2_data)
        significance_results.append({'Group 1': group1_name, 'Group 2': group2_name, 'Test': 'Mann-Whitney U Test', 'p-value': u_p_value})
        summary_results.append({'Group 1': group1_name, 'Group 2': group2_name, 'Significance Test': 'Mann-Whitney U Test', 'p-value': u_p_value, 'Significance': 'No' if u_p_value > 0.05 else 'Yes'})
        print(f"{group1_name}와 {group2_name} 간 Mann-Whitney U Test 결과: p-value = {u_p_value:.4f}")
else:
    # 그룹이 3개 이상인 경우 ANOVA 또는 Kruskal-Wallis H Test 수행
    group_names = list(valid_groups.keys())
    all_data = np.array(all_data)
    group_labels = np.array(group_labels)

    # 정규성 검증 결과에 따라 ANOVA 또는 Kruskal-Wallis H Test 선택
    normality_pass = all([result['Shapiro-Wilk p-value'] > 0.05 for result in normality_results if result['Group'] in group_names])

    if normality_pass:
        # 모든 그룹이 정규성을 만족하는 경우 ANOVA 수행
        f_stat, p_value = stats.f_oneway(*(valid_groups[group] for group in group_names))
        print(f"ANOVA 결과: p-value = {p_value:.4f}")
        if p_value <= 0.05:
            # 사후 검정 (Tukey's HSD)
            tukey_results = pairwise_tukeyhsd(all_data, group_labels)
            print(tukey_results)
            for result in tukey_results._results_table.data[1:]:
                significance_results.append({'Group 1': result[0], 'Group 2': result[1], 'Test': 'Tukey HSD', 'p-value': result[4], 'Significance': 'No' if result[4] > 0.05 else 'Yes'})
        else:
            print("그룹 간에 유의미한 차이가 없습니다. (p > 0.05)")
    else:
        # 하나라도 정규성을 만족하지 않는 경우 Kruskal-Wallis H Test 수행
        h_stat, p_value = stats.kruskal(*(valid_groups[group] for group in group_names))
        print(f"Kruskal-Wallis H Test 결과: p-value = {p_value:.4f}")
        if p_value <= 0.05:
            print("그룹 간에 유의미한 차이가 있습니다. 사후 검정 필요")
        else:
            print("그룹 간에 유의미한 차이가 없습니다. (p > 0.05)")

# 결과를 DataFrame으로 변환
normality_results_df = pd.DataFrame(normality_results)
significance_results_df = pd.DataFrame(significance_results)
summary_results_df = pd.DataFrame(summary_results)

# 엑셀 파일에 결과 저장 경로 설정
output_path = os.path.join(output_directory, "Result.xlsx")

# 엑셀 파일 작성 엔진을 명시적으로 지정
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    if not normality_results_df.empty:
        normality_results_df.to_excel(writer, sheet_name='Normality Test Results', index=False)
    if not significance_results_df.empty:
        significance_results_df.to_excel(writer, sheet_name='Significance Test Results', index=False)
    if not summary_results_df.empty:
        summary_results_df.to_excel(writer, sheet_name='Summary Results', index=False)

print(f"정규성 및 유의성 검증 결과가 {output_path}에 저장되었습니다.")


엑셀 파일이 성공적으로 로드하였습니다.
Other 그룹의 정규성 검증 (Shapiro-Wilk Test): p-value = 0.0000
Other 그룹은 정규 분포를 따르지 않습니다. (p <= 0.05)


ValueError: Need at least two groups in stats.kruskal()